In [ ]:
# ===============================
# BLIP: IMAGE CAPTIONING (BATCH PROCESSING)
# ===============================
import os
import pandas as pd
from pathlib import Path
from PIL import Image
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm.notebook import tqdm

# ---------- CONFIG ----------
import kagglehub
kaggle_root = kagglehub.dataset_download('nodoubttome/skin-cancer9-classesisic')
DATA_ROOT = os.path.join(
    kaggle_root,
    "Skin cancer ISIC The International Skin Imaging Collaboration",
    "Train"
)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8  # Adjust based on GPU memory

# ---------- LOAD IMAGE PATHS + LABELS ----------
image_paths, labels = [], []
root_path = Path(DATA_ROOT)

for label_dir in sorted(root_path.iterdir()):
    if label_dir.is_dir():
        for img_file in label_dir.iterdir():
            if img_file.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                image_paths.append(str(img_file))
                labels.append(label_dir.name)

df = pd.DataFrame({"image_path": image_paths, "label": labels})
print(f"✅ Loaded {len(df)} images across {df['label'].nunique()} classes.")

# ---------- LOAD BLIP MODEL ----------
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(DEVICE)

# ---------- GENERATE CAPTIONS IN BATCHES ----------
captions = []
for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Generating captions in batches"):
    batch_paths = df["image_path"].iloc[i:i+BATCH_SIZE].tolist()
    images = [Image.open(p).convert("RGB") for p in batch_paths]
    inputs = processor(images=images, return_tensors="pt").to(DEVICE)
    out = model.generate(**inputs)
    batch_captions = processor.batch_decode(out, skip_special_tokens=True)
    captions.extend(batch_captions)

df["caption"] = captions

# ---------- SHOW SAMPLE CAPTIONS ----------
print("\nSample Captions:")
print(df[["image_path", "label", "caption"]].head())

# ---------- SAVE CAPTIONS ----------
df.to_csv("captions_dataset.csv", index=False)
print("✅ Captions saved to captions_dataset.csv")


Using Colab cache for faster access to the 'skin-cancer9-classesisic' dataset.
✅ Loaded 2239 images across 9 classes.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating captions in batches:   0%|          | 0/280 [00:00<?, ?it/s]


Sample Captions:
                                          image_path              label  \
0  /kaggle/input/skin-cancer9-classesisic/Skin ca...  actinic keratosis   
1  /kaggle/input/skin-cancer9-classesisic/Skin ca...  actinic keratosis   
2  /kaggle/input/skin-cancer9-classesisic/Skin ca...  actinic keratosis   
3  /kaggle/input/skin-cancer9-classesisic/Skin ca...  actinic keratosis   
4  /kaggle/input/skin-cancer9-classesisic/Skin ca...  actinic keratosis   

                                   caption  
0            a close up of a piece of skin  
1           a close up of a pink substance  
2            a close up of a piece of skin  
3  a close up of a skin with a small patch  
4           a close up of a pink substance  
✅ Captions saved to captions_dataset.csv


In [ ]:
# ===============================
# FREE LLM CLASSIFIER (Flan-T5)
# ===============================
from transformers import pipeline
import json
from tqdm import tqdm

# Possible classes
POSSIBLE_LABELS = [
    "actinic keratosis", "basal cell carcinoma", "dermatofibroma", "melanoma",
    "nevus", "seborrheic keratosis", "squamous cell carcinoma", "vascular lesion", "normal"
]

# Define prompt
def make_prompt(caption):
    return f"""
You are a medical assistant.

Caption: "{caption}"

Possible labels: {POSSIBLE_LABELS}

Task:
1. Pick the most likely label from the list.
2. Give reasoning in 1-2 short sentences.

Respond in JSON:
{{"label": "...", "rationale": "..."}}
"""

# Load a free Hugging Face model (no token needed)
classifier = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",   # free + ungated
    device=0 if torch.cuda.is_available() else -1
)

# Run classification
predictions, rationales = [], []
for caption in tqdm(df["caption"].tolist(), desc="Classifying with Flan-T5"):
    response = classifier(make_prompt(caption), max_new_tokens=100)[0]["generated_text"]
    try:
        result = json.loads(response)
        predictions.append(result.get("label", "unknown"))
        rationales.append(result.get("rationale", ""))
    except:
        predictions.append(response.strip())
        rationales.append("raw output")

df["llm_prediction"] = predictions
df["llm_rationale"] = rationales

# Save results
df.to_csv("captions_with_predictions.csv", index=False)
print("✅ Predictions saved to captions_with_predictions.csv")

# Show sample
print(df[["caption", "llm_prediction", "llm_rationale"]].head())


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Classifying with Flan-T5: 100%|██████████| 2239/2239 [1:21:55<00:00,  2.20s/it]

✅ Predictions saved to captions_with_predictions.csv
                                   caption  \
0            a close up of a piece of skin   
1           a close up of a pink substance   
2            a close up of a piece of skin   
3  a close up of a skin with a small patch   
4           a close up of a pink substance   

                                      llm_prediction llm_rationale  
0  Seborrheic keratosis is a type of skin conditi...    raw output  
1  Seborrheic keratosis is a type of skin conditi...    raw output  
2  Seborrheic keratosis is a type of skin conditi...    raw output  
3  Seborrheic keratosis is a skin condition in wh...    raw output  
4  Seborrheic keratosis is a type of skin conditi...    raw output  


In [ ]:
sample_image_path = df["image_path"].iloc[0]  # just take the first image
print(sample_image_path)


/kaggle/input/skin-cancer9-classesisic/Skin cancer ISIC The International Skin Imaging Collaboration/Train/actinic keratosis/ISIC_0027580.jpg


In [ ]:
from PIL import Image

# Load image
image = Image.open(sample_image_path).convert("RGB")

# BLIP captioning
inputs = processor(images=image, return_tensors="pt").to(DEVICE)
out = model.generate(**inputs, max_new_tokens=64)
caption = processor.decode(out[0], skip_special_tokens=True)

print("Generated Caption:", caption)


Generated Caption: a close up of a piece of skin


In [ ]:
from transformers import pipeline
import torch, json

# Load free LLM
classifier = pipeline(
    "text2text-generation",
    model="google/flan-t5-large",
    device=0 if torch.cuda.is_available() else -1
)

# Possible labels
POSSIBLE_LABELS = [
    "actinic keratosis", "basal cell carcinoma", "dermatofibroma", "melanoma",
    "nevus", "seborrheic keratosis", "squamous cell carcinoma", "vascular lesion", "normal"
]

def make_prompt(caption):
    return f"""
You are a medical assistant.

Caption: "{caption}"

Possible labels: {POSSIBLE_LABELS}

Task:
1. Pick the most likely label from the list.
2. Give reasoning in 1–2 short sentences.

Respond in JSON:
{{"label": "...", "rationale": "..."}}
"""

# Run prediction
response = classifier(make_prompt(caption), max_new_tokens=100)[0]["generated_text"]

try:
    result = json.loads(response)
except:
    result = {"label": response.strip(), "rationale": "raw output"}

print("Predicted Label:", result["label"])
print("Rationale:", result["rationale"])


Device set to use cuda:0


Predicted Label: Seborrheic keratosis is a type of skin condition. Seborrheic keratosis is a type of skin condition. The answer: seborrheic keratosis.
Rationale: raw output


In [ ]:

# BLIP + CLASSIFIER (Accuracy Evaluation)

import os
import pandas as pd
from pathlib import Path
from PIL import Image
import numpy as np
import torch
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

from transformers import BlipProcessor, BlipModel

# ---------- CONFIG ----------
import kagglehub
kaggle_root = kagglehub.dataset_download('nodoubttome/skin-cancer9-classesisic')
DATA_ROOT = os.path.join(
    kaggle_root,
    "Skin cancer ISIC The International Skin Imaging Collaboration",
    "Train"
)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ---------- LOAD IMAGE PATHS + LABELS ----------
image_paths, labels = [], []
root_path = Path(DATA_ROOT)

for label_dir in sorted(root_path.iterdir()):
    if label_dir.is_dir():
        for img_file in label_dir.iterdir():
            if img_file.suffix.lower() in ['.png', '.jpg', '.jpeg']:
                image_paths.append(str(img_file))
                labels.append(label_dir.name)

df = pd.DataFrame({"image_path": image_paths, "label": labels})
print(f" Loaded {len(df)} images across {df['label'].nunique()} classes.")

# ---------- TRAIN/TEST SPLIT ----------
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

# ---------- LOAD BLIP MODEL (for embeddings) ----------
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipModel.from_pretrained("Salesforce/blip-image-captioning-base").to(DEVICE)
blip_model.eval()

# ---------- FEATURE EXTRACTION ----------
def get_blip_features(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        features = blip_model.get_image_features(**inputs)
    return features.cpu().numpy().flatten()

print(" Extracting BLIP embeddings ...")

X_train = np.array([get_blip_features(p) for p in tqdm(train_df["image_path"])])
y_train = train_df["label"].values

X_test = np.array([get_blip_features(p) for p in tqdm(test_df["image_path"])])
y_test = test_df["label"].values

print(" Feature extraction done:", X_train.shape, X_test.shape)

# ---------- TRAIN CLASSIFIER ----------
clf = LogisticRegression(max_iter=5000)
clf.fit(X_train, y_train)
print(" Training complete")

# ---------- EVALUATE ----------
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("\n Accuracy:", acc)
print("\n Classification Report:\n", classification_report(y_test, y_pred))


Using Colab cache for faster access to the 'skin-cancer9-classesisic' dataset.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ Loaded 2239 images across 9 classes.
Train: 1791, Test: 448


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

...

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Some weights of BlipModel were not initialized from the model checkpoint at Salesforce/blip-image-captioning-base and are newly initialized: ['logit_scale', 'text_model.embeddings.LayerNorm.bias', 'text_model.embeddings.LayerNorm.weight', 'text_model.embeddings.position_embeddings.weight', 'text_model.embeddings.word_embeddings.weight', 'text_model.encoder.layer.0.attention.output.LayerNorm.bias', 'text_model.encoder.layer.0.attention.output.LayerNorm.weight', 'text_model.encoder.layer.0.attention.output.dense.bias', 'text_model.encoder.layer.0.attention.output.dense.weight', 'text_model.encoder.layer.0.attention.self.key.bias', 'text_model.encoder.layer.0.attention.self.key.weight', 'text_model.encoder.layer.0.attention.self.query.bias', 'text_model.encoder.layer.0.attention.self.query.weight', 'text_model.encoder.layer.0.attention.self.value.bias', 'text_model.encoder.layer.0.attention.self.value.weight', 'text_model.encoder.layer.0.crossattention.output.LayerNorm.bias', 'text_model.

🔄 Extracting BLIP embeddings (this may take some time)...



100%|██████████| 448/448 [00:28<00:00, 15.51it/s]


✅ Feature extraction done: (1791, 512) (448, 512)
✅ Training complete

🎯 Accuracy: 0.6808035714285714

📊 Classification Report:
                             precision    recall  f1-score   support

         actinic keratosis       0.41      0.39      0.40        23
      basal cell carcinoma       0.70      0.77      0.73        75
            dermatofibroma       0.69      0.58      0.63        19
                  melanoma       0.68      0.74      0.71        88
                     nevus       0.63      0.65      0.64        71
pigmented benign keratosis       0.80      0.81      0.80        93
      seborrheic keratosis       0.00      0.00      0.00        15
   squamous cell carcinoma       0.53      0.44      0.48        36
           vascular lesion       0.86      0.89      0.88        28

                  accuracy                           0.68       448
                 macro avg       0.59      0.59      0.59       448
              weighted avg       0.66      0.68      